In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

In [13]:
# Load your CSV file
# Assuming the CSV file contains features and target values (X, y)
data = pd.read_csv('coords-cleaned-nohands-rmv - Copy.csv')

# Separate features (X) and target values (y)
X = data.drop('class', axis=1).values
y = data['class'].values

#y = np.array(y, dtype=np.float32)


In [14]:
# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape X if it's a 1D array
# X = X.reshape(X.shape[0], X.shape[1], 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:

X_train = torch.from_numpy(scaler.fit_transform(X_train.numpy())).float()
y_train = torch.from_numpy(y_train).long().view(-1)  # Use .view(-1) to flatten

X_test = torch.from_numpy(scaler.transform(X_test.numpy())).float()
y_test = torch.from_numpy(y_test).long().view(-1)    # Use .view(-1) to flatten



TypeError: expected np.ndarray (got Tensor)

In [16]:
# Build the CNN model using PyTorch
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 32, kernel_size=3)
        self.pool = nn.MaxPool1d(2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32 * ((X_train.shape[1]-2)//2), 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# Instantiate the model, loss function, and optimizer
model = SimpleCNN()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

# Training loop
epochs = 10
train_losses = []
test_losses = []
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train.unsqueeze(1))
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    train_losses.append(loss.item())

    # Evaluate on the test set
    model.eval()
    with torch.no_grad():
        test_outputs = model(X_test.unsqueeze(1))
        test_loss = criterion(test_outputs, y_test)
        test_losses.append(test_loss.item())

# Plot Mean Squared Error graph
plt.plot(train_losses, label='Training Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Mean Squared Error')
plt.legend()
plt.show()


RuntimeError: 0D or 1D target tensor expected, multi-target not supported